In [2]:
# 📥 Cargar dataset base
import pandas as pd

df = pd.read_csv("dataset_base.csv", low_memory=False)
print("Shape:", df.shape)
print("Columnas:")
print(df.columns.tolist())

# Vista rápida
df.head()


Shape: (9460980, 13)
Columnas:
['customer_id', 'product_id', 'periodo', 'plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'tn', 'stock_final', 'cat1', 'cat2', 'cat3', 'brand', 'sku_size']


,customer_id,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,stock_final,cat1,cat2,cat3,brand,sku_size
0,10004,20001,2017-01,0.0,9.0,184.72927,184.72927,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000
1,10003,20001,2017-01,0.0,17.0,143.49426,143.49426,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000
2,10002,20001,2017-01,0.0,17.0,38.68301,35.72806,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000
3,10001,20001,2017-01,0.0,11.0,99.43861,99.43861,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000
4,10084,20001,2017-01,0.0,1.0,0.22386,0.22386,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000


In [4]:
# ✅ Transformación de periodo
df['periodo'] = pd.to_datetime(df['periodo'], format='%Y-%m')


In [5]:
# 📊 Normalización estandar por producto
from sklearn.preprocessing import StandardScaler

scaler_dict = {}
for col in ['tn', 'cust_request_tn']:
    df[f'{col}_norm'] = 0.0
    for prod in df['product_id'].unique():
        idx = df['product_id'] == prod
        scaler = StandardScaler()
        df.loc[idx, f'{col}_norm'] = scaler.fit_transform(df.loc[idx, [col]])
        scaler_dict[(col, prod)] = scaler


In [6]:
# 🧩 Codificación de variables categóricas

from sklearn.preprocessing import LabelEncoder

# Asegurar que los identificadores estén marcados como categorías
df['product_id'] = df['product_id'].astype('category')
df['customer_id'] = df['customer_id'].astype('category')

# Codificar variables categóricas textuales si las hay
for col in ['cat1', 'cat2', 'cat3', 'brand']:
    df[col] = df[col].astype(str)  # por las dudas, si hay algún NaN o numérico disfrazado
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])


In [7]:
df.shape

(9460980, 15)

In [8]:
print(df.columns.tolist())


['customer_id', 'product_id', 'periodo', 'plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'tn', 'stock_final', 'cat1', 'cat2', 'cat3', 'brand', 'sku_size', 'tn_norm', 'cust_request_tn_norm']


In [9]:
# 🛠 Feature Engineering extendida
import numpy as np

# Asegurarse de tener los datos ordenados por producto y periodo
df = df.sort_values(['product_id', 'periodo'])

# --- 1. Lags de tn: desde 1 a 36
for i in range(1, 37):
    df[f'lag_{i}'] = df.groupby('product_id')['tn'].shift(i)

# --- 2. Delta lags: diferencia entre lags consecutivos
for i in range(1, 36):
    df[f'delta_lag_{i}'] = df[f'lag_{i}'] - df[f'lag_{i+1}']

# --- 3. Medias móviles: rolling mean y std
for win in [3, 6, 12, 24, 36]:
    df[f'tn_mean_{win}'] = df.groupby('product_id')['tn'].transform(lambda x: x.rolling(win, min_periods=1).mean())
    df[f'tn_std_{win}'] = df.groupby('product_id')['tn'].transform(lambda x: x.rolling(win, min_periods=1).std())

# --- 4. Mínimos booleanos: 1 si el valor es el mínimo en su ventana
for win in [3, 6, 12, 24, 36]:
    roll_min = df.groupby('product_id')['tn'].transform(lambda x: x.rolling(win, min_periods=1).min())
    df[f'is_min_{win}'] = (df['tn'] == roll_min).astype(int)

# --- 5. Máximos booleanos: 1 si el valor es el máximo en su ventana
for win in [3, 6, 12, 24, 36]:
    roll_max = df.groupby('product_id')['tn'].transform(lambda x: x.rolling(win, min_periods=1).max())
    df[f'is_max_{win}'] = (df['tn'] == roll_max).astype(int)

# --- 6. Features temporales
df['año'] = df['periodo'].dt.year
df['mes'] = df['periodo'].dt.month
df['cuatrimestre'] = ((df['mes'] - 1) // 4) + 1

# Continuidad temporal (número de mes absoluto desde el inicio)
min_fecha = df['periodo'].min()
df['mes_absoluto'] = ((df['periodo'].dt.year - min_fecha.year) * 12 +
                      (df['periodo'].dt.month - min_fecha.month))

# Coseno del mes para estacionalidad
df['cos_mes'] = np.cos(2 * np.pi * df['mes'] / 12)


C:\Users\waldo\AppData\Local\Temp\ipykernel_44800\596090252.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[f'lag_{i}'] = df.groupby('product_id')['tn'].shift(i)
C:\Users\waldo\AppData\Local\Temp\ipykernel_44800\596090252.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[f'lag_{i}'] = df.groupby('product_id')['tn'].shift(i)
C:\Users\waldo\AppData\Local\Temp\ipykernel_44800\596090252.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default an

In [10]:
df.shape

(9460980, 111)

In [11]:
print(df.columns.tolist())


['customer_id', 'product_id', 'periodo', 'plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'tn', 'stock_final', 'cat1', 'cat2', 'cat3', 'brand', 'sku_size', 'tn_norm', 'cust_request_tn_norm', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'lag_8', 'lag_9', 'lag_10', 'lag_11', 'lag_12', 'lag_13', 'lag_14', 'lag_15', 'lag_16', 'lag_17', 'lag_18', 'lag_19', 'lag_20', 'lag_21', 'lag_22', 'lag_23', 'lag_24', 'lag_25', 'lag_26', 'lag_27', 'lag_28', 'lag_29', 'lag_30', 'lag_31', 'lag_32', 'lag_33', 'lag_34', 'lag_35', 'lag_36', 'delta_lag_1', 'delta_lag_2', 'delta_lag_3', 'delta_lag_4', 'delta_lag_5', 'delta_lag_6', 'delta_lag_7', 'delta_lag_8', 'delta_lag_9', 'delta_lag_10', 'delta_lag_11', 'delta_lag_12', 'delta_lag_13', 'delta_lag_14', 'delta_lag_15', 'delta_lag_16', 'delta_lag_17', 'delta_lag_18', 'delta_lag_19', 'delta_lag_20', 'delta_lag_21', 'delta_lag_22', 'delta_lag_23', 'delta_lag_24', 'delta_lag_25', 'delta_lag_26', 'delta_lag_27', 'delta_lag_28', 'del

In [13]:
# 🔐 Guardar dataset procesado para no perder avances
df.to_parquet("df_features_111.parquet", index=False)
print("✅ Dataset guardado como df_features_111.parquet")


✅ Dataset guardado como df_features_111.parquet


In [14]:
# 📦 Cargar dataset previamente guardado
import pandas as pd

df = pd.read_parquet("df_features_111.parquet")
print(f"✅ Dataset cargado con shape: {df.shape}")


✅ Dataset cargado con shape: (9460980, 111)


In [ ]:
# 💾 Guardar dataset extendido
#df.to_csv("dataset_base_features.csv", index=False)


In [15]:
# 🤝 Clustering con Dynamic Time Warping (DTW)
# Requiere tslearn (pip install tslearn)

from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
import numpy as np

# 1. Crear matriz de series temporales por producto
df_dtw = df[df['tn'] > 0].copy()
serie = df_dtw.pivot_table(index='product_id', columns='periodo', values='tn', aggfunc='sum').fillna(0)
serie_array = serie.values[:, :, np.newaxis]  # shape (n_series, n_timestamps, 1)
serie_scaled = TimeSeriesScalerMeanVariance().fit_transform(serie_array)

# 2. Clustering con DTW
modelo_dtw = TimeSeriesKMeans(n_clusters=50, metric="dtw", random_state=42)
labels_dtw = modelo_dtw.fit_predict(serie_scaled)

# 3. Agregar etiquetas de cluster al dataset principal
df_clusters = pd.DataFrame({'product_id': serie.index, 'cluster_dtw': labels_dtw})
df['product_id'] = df['product_id'].astype(str)
df_clusters['product_id'] = df_clusters['product_id'].astype(str)
df = df.merge(df_clusters, on='product_id', how='left')


c:\Users\waldo\miniconda3\envs\tslearn_env\lib\site-packages\tslearn\bases\bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)
c:\Users\waldo\miniconda3\envs\tslearn_env\lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\waldo\miniconda3\envs\tslearn_env\lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\waldo\miniconda3\envs\tslearn_env\lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\waldo\miniconda3\envs\tslearn_env\lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all

In [21]:
from reducirDT import optimize_memory_usage
df = optimize_memory_usage(df)

MemoryError: Unable to allocate 6.27 GiB for an array with shape (89, 9460980) and data type float64

In [22]:
from prophet import Prophet
import pandas as pd
import os
from tqdm import tqdm

# ⚙️ Configuración
ARCHIVO_SALIDA = "df_prophet_features"
CARPETA_SALIDA = "prophet_features_parciales"
os.makedirs(CARPETA_SALIDA, exist_ok=True)

# Lista de productos
productos = df['product_id'].astype(str).unique()

# Filtrar los ya procesados
procesados = {f.split('.')[0] for f in os.listdir(CARPETA_SALIDA)}
productos_faltantes = [p for p in productos if p not in procesados]

print(f"👉 Hay {len(productos_faltantes)} productos por procesar...")

# 🚀 Loop por producto
for prod_id in tqdm(productos_faltantes):
    grupo = df[df['product_id'] == prod_id].copy()
    df_prophet = grupo[['periodo', 'tn']].rename(columns={'periodo': 'ds', 'tn': 'y'})
    df_prophet = df_prophet[df_prophet['y'] > 0]

    if len(df_prophet) < 12:
        continue  # muy poco histórico

    try:
        modelo = Prophet(daily_seasonality=False, weekly_seasonality=False, yearly_seasonality=True)
        modelo.fit(df_prophet)
        forecast = modelo.predict(df_prophet)

        componentes = forecast[['ds', 'trend']]
        componentes.columns = ['periodo', 'trend_prophet']
        if 'seasonal' in forecast.columns:
            componentes['seasonal_prophet'] = forecast['seasonal']
        else:
            componentes['seasonal_prophet'] = 0

        resultado = grupo.merge(componentes, on='periodo', how='left')
        resultado.to_parquet(f"{CARPETA_SALIDA}/{prod_id}.parquet", index=False)

    except Exception as e:
        print(f"❌ Error con {prod_id}: {e}")


👉 Hay 780 productos por procesar...


  0%|          | 0/780 [00:00<?, ?it/s]18:40:13 - cmdstanpy - INFO - Chain [1] start processing
18:40:14 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\waldo\AppData\Local\Temp\ipykernel_44800\1165039906.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  componentes['seasonal_prophet'] = 0
  0%|          | 1/780 [00:15<3:15:55, 15.09s/it]18:40:29 - cmdstanpy - INFO - Chain [1] start processing
18:40:29 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\waldo\AppData\Local\Temp\ipykernel_44800\1165039906.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

In [2]:
import pandas as pd
import glob
import os
from tqdm import tqdm

carpeta_parquets = "prophet_features_parciales"
archivos = sorted(glob.glob(os.path.join(carpeta_parquets, "*.parquet")))
salida = "df_features_prophet_final.parquet"
checkpoint_file = "archivos_procesados.txt"

# Inicializar checkpoint
procesados = set()
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, "r") as f:
        procesados = set(line.strip() for line in f)

# Si existe salida vieja, eliminarla para evitar duplicaciones
if not os.path.exists(salida):
    primer_escritura = True
else:
    primer_escritura = False

print(f"🔁 Procesados previamente: {len(procesados)}")

for archivo in tqdm(archivos, desc="📦 Procesando archivos"):
    if archivo in procesados:
        continue
    try:
        df = pd.read_parquet(archivo)
        df.to_parquet(salida, index=False, append=not primer_escritura)
        primer_escritura = False

        with open(checkpoint_file, "a") as f:
            f.write(archivo + "\n")

    except Exception as e:
        print(f"⚠️ Error al procesar {archivo}: {e}")


🔁 Procesados previamente: 85


📦 Procesando archivos:   9%|▉         | 72/780 [00:00<00:09, 74.01it/s]

⚠️ Error al procesar prophet_features_parciales\20079.parquet: __cinit__() got an unexpected keyword argument 'append'


📦 Procesando archivos:  11%|█         | 82/780 [00:01<00:13, 53.06it/s]

⚠️ Error al procesar prophet_features_parciales\20091.parquet: __cinit__() got an unexpected keyword argument 'append'
⚠️ Error al procesar prophet_features_parciales\20095.parquet: __cinit__() got an unexpected keyword argument 'append'


...

⚠️ Error al procesar prophet_features_parciales\21226.parquet: __cinit__() got an unexpected keyword argument 'append'
⚠️ Error al procesar prophet_features_parciales\21227.parquet: __cinit__() got an unexpected keyword argument 'append'
⚠️ Error al procesar prophet_features_parciales\21233.parquet: __cinit__() got an unexpected keyword argument 'append'
⚠️ Error al procesar prophet_features_parciales\21244.parquet: __cinit__() got an unexpected keyword argument 'append'


📦 Procesando archivos:  99%|█████████▉| 772/780 [05:19<00:00, 12.25it/s]

⚠️ Error al procesar prophet_features_parciales\21245.parquet: __cinit__() got an unexpected keyword argument 'append'
⚠️ Error al procesar prophet_features_parciales\21246.parquet: __cinit__() got an unexpected keyword argument 'append'
⚠️ Error al procesar prophet_features_parciales\21248.parquet: __cinit__() got an unexpected keyword argument 'append'
⚠️ Error al procesar prophet_features_parciales\21252.parquet: __cinit__() got an unexpected keyword argument 'append'


📦 Procesando archivos: 100%|██████████| 780/780 [05:19<00:00,  2.44it/s]

⚠️ Error al procesar prophet_features_parciales\21256.parquet: __cinit__() got an unexpected keyword argument 'append'
⚠️ Error al procesar prophet_features_parciales\21259.parquet: __cinit__() got an unexpected keyword argument 'append'
⚠️ Error al procesar prophet_features_parciales\21262.parquet: __cinit__() got an unexpected keyword argument 'append'
⚠️ Error al procesar prophet_features_parciales\21263.parquet: __cinit__() got an unexpected keyword argument 'append'
⚠️ Error al procesar prophet_features_parciales\21265.parquet: __cinit__() got an unexpected keyword argument 'append'
⚠️ Error al procesar prophet_features_parciales\21266.parquet: __cinit__() got an unexpected keyword argument 'append'
⚠️ Error al procesar prophet_features_parciales\21267.parquet: __cinit__() got an unexpected keyword argument 'append'
⚠️ Error al procesar prophet_features_parciales\21276.parquet: __cinit__() got an unexpected keyword argument 'append'


In [5]:
# ⚙️ Entrenamiento con XGBoost + sample_weight
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# 1. Eliminar filas con NaNs en features (por rolling, lags, etc.)
df_modelo = df.dropna()

# 2. Definir columnas a excluir
cols_excluir = ['periodo', 'tn', 'product_id', 'customer_id']
X = df_modelo.drop(columns=cols_excluir)
y = df_modelo['tn']

# 3. Crear sample_weight en base al promedio del producto (opcional pero común)
promedios = df_modelo.groupby('product_id')['tn'].transform('mean')
sample_weight = df_modelo['tn'] / promedios
sample_weight = sample_weight.fillna(1.0)

# 4. División temporal
X_train = X[df_modelo['periodo'] < '2019-12-01']
X_val   = X[df_modelo['periodo'] == '2019-12-01']
y_train = y[df_modelo['periodo'] < '2019-12-01']
y_val   = y[df_modelo['periodo'] == '2019-12-01']
w_train = sample_weight[df_modelo['periodo'] < '2019-12-01']
w_val   = sample_weight[df_modelo['periodo'] == '2019-12-01']

# 5. Entrenar modelo
modelo = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=300,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    tree_method='hist'
)

modelo.fit(X_train, y_train, sample_weight=w_train, eval_set=[(X_val, y_val)],verbose=True)
           #early_stopping_rounds=30, 

# 6. Evaluación
y_pred = modelo.predict(X_val)
mae = mean_absolute_error(y_val, y_pred)
print(f"📊 MAE validación diciembre 2019: {mae:.3f}")


[0]	validation_0-rmse:0.02945
[1]	validation_0-rmse:0.02681
[2]	validation_0-rmse:0.02441
[3]	validation_0-rmse:0.02227
[4]	validation_0-rmse:0.02028
[5]	validation_0-rmse:0.01853
[6]	validation_0-rmse:0.01686
[7]	validation_0-rmse:0.01539
[8]	validation_0-rmse:0.01404
[9]	validation_0-rmse:0.01279
[10]	validation_0-rmse:0.01167
[11]	validation_0-rmse:0.01067
[12]	validation_0-rmse:0.00976
[13]	validation_0-rmse:0.00898
[14]	validation_0-rmse:0.00824
[15]	validation_0-rmse:0.00755
[16]	validation_0-rmse:0.00692
[17]	validation_0-rmse:0.00639
[18]	validation_0-rmse:0.00588
[19]	validation_0-rmse:0.00541
[20]	validation_0-rmse:0.00499
[21]	validation_0-rmse:0.00463
[22]	validation_0-rmse:0.00429
[23]	validation_0-rmse:0.00400
[24]	validation_0-rmse:0.00372
[25]	validation_0-rmse:0.00343
[26]	validation_0-rmse:0.00321
[27]	validation_0-rmse:0.00300
[28]	validation_0-rmse:0.00278
[29]	validation_0-rmse:0.00258
[30]	validation_0-rmse:0.00243
[31]	validation_0-rmse:0.00226
[32]	validation_0-

In [6]:
# ⚙️ Entrenamiento final con los mejores hiperparámetros de Optuna
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

# 1. Extraer los mejores parámetros de Optuna y completarlos si hace falta
params_optimos = study.best_params
params_optimos.update({
    'objective': 'reg:squarederror',
    'tree_method': 'hist',
    'random_state': 42
})

# 2. Entrenar modelo final
modelo_final = xgb.XGBRegressor(**params_optimos)

modelo_final.fit(
    X_train, y_train,
    sample_weight=w_train,
    eval_set=[(X_val, y_val)],
    early_stopping_rounds=30,
    verbose=True
)

# 3. Evaluar
y_pred_final = modelo_final.predict(X_val)
mae_final = mean_absolute_error(y_val, y_pred_final)
print(f"📊 MAE validación diciembre 2019 (modelo final): {mae_final:.3f}")


NameError: name 'study' is not defined

In [7]:
# 🔮 Predicción para enero y febrero 2020

# 1. Obtener los datos de diciembre 2019 para generar lags
df_2019_12 = df[df['periodo'] == '2019-12-01'].copy()
df_2019_11 = df[df['periodo'] == '2019-11-01'].copy()
df_2019_10 = df[df['periodo'] == '2019-10-01'].copy()

# 2. Predicción enero 2020
enero = df_2019_12.copy()
enero['periodo'] = pd.to_datetime('2020-01-01')
enero['lag_1'] = df_2019_12['tn']
enero['lag_2'] = df_2019_11['tn'].values
enero['lag_3'] = df_2019_10['tn'].values
enero['mes'] = 1
enero['mes_absoluto'] = df_2019_12['mes_absoluto'].max() + 1
enero['cos_mes'] = np.cos(2 * np.pi * 1 / 12)

X_enero = enero[X_val.columns]
enero['tn_pred'] = modelo_final.predict(X_enero)

# 3. Predicción febrero 2020
febrero = enero.copy()
febrero['periodo'] = pd.to_datetime('2020-02-01')
febrero['lag_1'] = enero['tn_pred']
febrero['lag_2'] = df_2019_12['tn'].values
febrero['lag_3'] = df_2019_11['tn'].values
febrero['mes'] = 2
febrero['mes_absoluto'] += 1
febrero['cos_mes'] = np.cos(2 * np.pi * 2 / 12)

X_febrero = febrero[X_val.columns]
febrero['tn_predicho'] = modelo_final.predict(X_febrero)

# 4. Guardar resultados
resultado = febrero[['product_id', 'tn_predicho']]
resultado.to_csv("prediccion_febrero2020_xgb.csv", index=False)
print("✅ Archivo generado: prediccion_febrero2020_xgb.csv")


ValueError: Length of values (184) does not match length of index (92)

In [10]:
# 🔍 Búsqueda de hiperparámetros con Optuna
# Requiere: pip install optuna

import optuna
from sklearn.metrics import mean_absolute_error
import xgboost as xgb

# 1. Definición de la función objetivo
def objective(trial):
    params = {
        'objective': 'reg:squarederror',
        'tree_method': 'hist',
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        'random_state': 42
    }

    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train, sample_weight=w_train,
              eval_set=[(X_val, y_val)],
              #early_stopping_rounds=30,
              verbose=False)
    
    preds = model.predict(X_val)
    score = mean_absolute_error(y_val, preds)
    return score

# 2. Ejecutar la optimización
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# 3. Mostrar los mejores parámetros
print("Mejores parámetros:")
print(study.best_params)


[I 2025-06-26 01:06:47,603] A new study created in memory with name: no-name-9212ec9d-4d9c-43ca-a6bc-a0914fa44ffe


[I 2025-06-26 01:06:48,735] Trial 0 finished with value: 0.020677971569817998 and parameters: {'n_estimators': 820, 'max_depth': 5, 'learning_rate': 0.22564130397480178, 'min_child_weight': 5, 'subsample': 0.5800119333843776, 'colsample_bytree': 0.6825839564660157, 'reg_alpha': 6.056399567797762, 'reg_lambda': 0.9127312040832638}. Best is trial 0 with value: 0.020677971569817998.
[I 2025-06-26 01:06:48,957] Trial 1 finished with value: 0.009629573346233886 and parameters: {'n_estimators': 110, 'max_depth': 20, 'learning_rate': 0.05702238198183828, 'min_child_weight': 8, 'subsample': 0.9535248876450441, 'colsample_bytree': 0.7591112636080086, 'reg_alpha': 2.3229167236580883, 'reg_lambda': 0.9472821342683652}. Best is trial 1 with value: 0.009629573346233886.
[I 2025-06-26 01:06:50,029] Trial 2 finished with value: 0.01521912301890228 and parameters: {'n_estimators': 845, 'max_depth': 14, 'learning_rate': 0.13308924395279467, 'min_child_weight': 10, 'subsample': 0.6980505987213735, 'cols

Mejores parámetros:
{'n_estimators': 853, 'max_depth': 20, 'learning_rate': 0.27349312955835653, 'min_child_weight': 6, 'subsample': 0.607613208566985, 'colsample_bytree': 0.7571434469577842, 'reg_alpha': 0.011498811736107943, 'reg_lambda': 0.4509749112848259}


In [12]:
# ⚙️ Entrenamiento final con los mejores hiperparámetros de Optuna
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

# 1. Extraer los mejores parámetros de Optuna y completarlos si hace falta
params_optimos = study.best_params
params_optimos.update({
    'objective': 'reg:squarederror',
    'tree_method': 'hist',
    'random_state': 42
})

# 2. Entrenar modelo final
modelo_final = xgb.XGBRegressor(**params_optimos)

modelo_final.fit(
    X_train, y_train,
    sample_weight=w_train,
    eval_set=[(X_val, y_val)],
    #early_stopping_rounds=30,
    verbose=True
)

# 3. Evaluar
y_pred_final = modelo_final.predict(X_val)
mae_final = mean_absolute_error(y_val, y_pred_final)
print(f"📊 MAE validación diciembre 2019 (modelo final): {mae_final:.3f}")


[0]	validation_0-rmse:0.02437
[1]	validation_0-rmse:0.01850
[2]	validation_0-rmse:0.01408
[3]	validation_0-rmse:0.01109
[4]	validation_0-rmse:0.00877
[5]	validation_0-rmse:0.00684
[6]	validation_0-rmse:0.00546
[7]	validation_0-rmse:0.00462
[8]	validation_0-rmse:0.00380
[9]	validation_0-rmse:0.00335
[10]	validation_0-rmse:0.00300
[11]	validation_0-rmse:0.00257
[12]	validation_0-rmse:0.00231
[13]	validation_0-rmse:0.00215
[14]	validation_0-rmse:0.00195
[15]	validation_0-rmse:0.00188
[16]	validation_0-rmse:0.00179
[17]	validation_0-rmse:0.00168
[18]	validation_0-rmse:0.00164
[19]	validation_0-rmse:0.00162
[20]	validation_0-rmse:0.00160
[21]	validation_0-rmse:0.00157
[22]	validation_0-rmse:0.00157
[23]	validation_0-rmse:0.00152
[24]	validation_0-rmse:0.00151
[25]	validation_0-rmse:0.00151
[26]	validation_0-rmse:0.00151
[27]	validation_0-rmse:0.00151
[28]	validation_0-rmse:0.00149
[29]	validation_0-rmse:0.00149
[30]	validation_0-rmse:0.00149
[31]	validation_0-rmse:0.00149
[32]	validation_0-

In [14]:
enero = enero.merge(df_2019_11[['product_id', 'tn']], on='product_id', how='left')
enero.rename(columns={'tn': 'lag_2'}, inplace=True)


In [16]:
print("Enero:", enero.shape)
print("Productos únicos en enero:", enero['product_id'].nunique())

print("Diciembre:", df_2019_12.shape)
print("Productos únicos en diciembre:", df_2019_12['product_id'].nunique())

print("Noviembre:", df_2019_11.shape)
print("Productos únicos en noviembre:", df_2019_11['product_id'].nunique())

print("Duplicados en df_2019_11:", df_2019_11.duplicated('product_id').sum())


Enero: (92, 114)
Productos únicos en enero: 1
Diciembre: (92, 114)
Productos únicos en diciembre: 1
Noviembre: (184, 114)
Productos únicos en noviembre: 1
Duplicados en df_2019_11: 183


In [15]:
# 🔮 Predicción para enero y febrero 2020

# 1. Obtener los datos de diciembre 2019 para generar lags
df_2019_12 = df[df['periodo'] == '2019-12-01'].copy()
df_2019_11 = df[df['periodo'] == '2019-11-01'].copy()
df_2019_10 = df[df['periodo'] == '2019-10-01'].copy()

# 2. Predicción enero 2020
enero = df_2019_12.copy()
enero['periodo'] = pd.to_datetime('2020-01-01')
enero['lag_1'] = df_2019_12['tn']
enero['lag_2'] = df_2019_11['tn'].values
enero['lag_3'] = df_2019_10['tn'].values
enero['mes'] = 1
enero['mes_absoluto'] = df_2019_12['mes_absoluto'].max() + 1
enero['cos_mes'] = np.cos(2 * np.pi * 1 / 12)

X_enero = enero[X_val.columns]
enero['tn_pred'] = modelo_final.predict(X_enero)

# 3. Predicción febrero 2020
febrero = enero.copy()
febrero['periodo'] = pd.to_datetime('2020-02-01')
febrero['lag_1'] = enero['tn_pred']
febrero['lag_2'] = df_2019_12['tn'].values
febrero['lag_3'] = df_2019_11['tn'].values
febrero['mes'] = 2
febrero['mes_absoluto'] += 1
febrero['cos_mes'] = np.cos(2 * np.pi * 2 / 12)

X_febrero = febrero[X_val.columns]
febrero['tn_predicho'] = modelo_final.predict(X_febrero)

# 4. Guardar resultados
resultado = febrero[['product_id', 'tn_predicho']]
resultado.to_csv("prediccion_febrero2020_xgb.csv", index=False)
print("✅ Archivo generado: prediccion_febrero2020_xgb.csv")


ValueError: Length of values (184) does not match length of index (92)

In [ ]:
# 📦 Preparación del archivo de envío Kaggle

# 1. Si tenés un archivo template de Kaggle, lo cargás así:
# df_template = pd.read_csv("submission_template.csv")
# ids_kaggle = df_template['product_id']

# Pero si no tenés un template y sólo querés asegurar un buen CSV:
# 2. Asegurar que haya 1 fila por product_id (completar faltantes)
todos_los_productos = df['product_id'].drop_duplicates()
resultado_final = pd.DataFrame({'product_id': todos_los_productos})

# 3. Merge con predicciones
resultado = pd.read_csv("prediccion_febrero2020_xgb.csv")
resultado_final = resultado_final.merge(resultado, on='product_id', how='left')

# 4. Si hay faltantes, imputarlos (opcional)
# estrategia: 0 o media histórica del producto
resultado_final['tn_predicho'] = resultado_final['tn_predicho'].fillna(0)

# 5. Exportar CSV listo para envío
resultado_final.to_csv("submission_kaggle_feb2020.csv", index=False)
print("📤 Archivo generado para Kaggle: submission_kaggle_feb2020.csv")


In [ ]:
# 📈 Comparación TN real vs TN predicha para diciembre 2019
import matplotlib.pyplot as plt
import pandas as pd

# Crear DataFrame de comparación
df_eval = pd.DataFrame({
    'product_id': X_val.index,
    'tn_real': y_val,
    'tn_predicha': y_pred_final
})

# Agrupar por producto si hay múltiples filas
df_eval_grouped = df_eval.groupby('product_id').mean()

# Graficar
plt.figure(figsize=(8, 6))
plt.scatter(df_eval_grouped['tn_real'], df_eval_grouped['tn_predicha'], alpha=0.5)
plt.plot([0, df_eval_grouped['tn_real'].max()],
         [0, df_eval_grouped['tn_real'].max()],
         'r--', label='Ideal')

plt.xlabel('TN real (diciembre 2019)')
plt.ylabel('TN predicha')
plt.title('Comparación por producto: TN real vs TN predicha')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# 🕒 División antes de más transformación
#df = df[df['periodo'] < '2020-01-01']  # o lo que corresponda al último mes del set
